# LLMCheckerChain
This notebook showcases how to use LLMCheckerChain.

In [1]:
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.llms import OpenAI

from langchain.prompts.prompt import PromptTemplate

llm = OpenAI(temperature=0.7)

get_name = LLMChain(
    llm=llm,
    prompt=PromptTemplate(input_variables=["gender"], template="Question: Give me a totally random first name for a {gender}. Answer:"),
    n=2,
)
# create_draft_answer_chain.run("woman")

get_initials = LLMChain(
    llm=llm,
    prompt=PromptTemplate(input_variables=["name"], template="Question: Give me a totally random last name for someone with the first name '{name}', with alliteration. Answer (full name):"),
    n=2,
)

overall_chain = SimpleSequentialChain(chains=[get_name, get_initials], verbose=True)
overall_chain.run("woman")



> Entering new SimpleSequentialChain chain...
responses
['\n\nMarielle', '\n\nMelanie']
['\n\nMarielle', '\n\nMelanie']
responses
[' Marielle Mallow', ' Marielle Mackenzie.', ' Melanie Mowatt', ' Melanie MacMillan']
[' Marielle Mallow', ' Marielle Mackenzie.', ' Melanie Mowatt', ' Melanie MacMillan']

> Finished SimpleSequentialChain chain.


[' Marielle Mallow',
 ' Marielle Mackenzie.',
 ' Melanie Mowatt',
 ' Melanie MacMillan']

In [2]:
llm = OpenAI(temperature=0.7)

get_description = LLMChain(
    llm=llm,
    prompt=PromptTemplate(input_variables=["persona"], template="A description of {persona}:"),
    output_key='description',
    n=2,
)

get_initials = LLMChain(
    llm=llm,
    prompt=PromptTemplate(input_variables=["description", "question"], template="Here's a description of a character: ```{description}```\n\nHow would this character answer the question '{question}'?\n\nAnswer:"),
    output_key='answer',
    n=2,
)

overall_chain = SequentialChain(chains=[get_description, get_initials], input_variables=['persona', 'question'], output_variables=['answer'], verbose=True)
overall_chain({'question': 'Who is the CEO of Chartbeat?', 'persona': 'a professional fact-checker'})



> Entering new SequentialChain chain...
responses
['\n\nA professional fact-checker is someone who is responsible for verifying the accuracy of written material before it is published. They are meticulous researchers who use a variety of sources, including databases, interviews, and published documents, to verify facts. They also have excellent writing, communication, and organizational skills, as well as a keen eye for detail. They must be able to identify and correct errors in text, as well as recognize potential problems with content. Fact-checkers must be able to work under tight deadlines and must be able to communicate any discrepancies or issues to the editor.', '\n\nA professional fact-checker is someone who verifies the accuracy of information in a variety of mediums, including articles, books, television shows, and other sources of information. They use a variety of methods to assess the accuracy of sources, such as reading original documents, cross-checking references, and

{'question': 'Who is the CEO of Chartbeat?',
 'persona': 'a professional fact-checker',
 'answer': [" I'm sorry, but I'm not familiar with the answer to that question. As a professional fact-checker, I specialize in verifying the accuracy of written material, not executive roles in specific companies.",
  " I'm sorry, I'm not sure who the CEO of Chartbeat is. However, I am confident I can verify the answer if you provide me with some reliable sources.",
  ' The CEO of Chartbeat is John Saroff.',
  ' After researching the company, I have determined that John Saroff is the CEO of Chartbeat.']}